<a href="https://colab.research.google.com/github/AK-2406/Major-Project-/blob/main/YOLO%26OCR_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 7.8 MB/s eta 0:00:00


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/905.3 kB 26.6 MB/s eta 0:00:00


In [3]:
!pip install Pillow

In [4]:
from google.colab import files
uploaded = files.upload()

Saving best.pt to best.pt


In [5]:
from google.colab import files
uploaded = files.upload()

Saving test_video.mp4 to test_video.mp4


In [ ]:
import os
import torch
import easyocr
import cv2
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors
from google.colab.patches import cv2_imshow
from PIL import Image

# Load your trained YOLO model
model = YOLO('best.pt')  # Replace with the path to your trained model

# Initialize EasyOCR Reader
reader = easyocr.Reader(['en'])  # Add more languages if needed

# OCR Function (remains the same)
def getOCR(im, coors):
    x, y, w, h = int(coors[0]), int(coors[1]), int(coors[2]), int(coors[3])
    cropped = im[y:h, x:w]
    gray = cv2.cvtColor(cropped, cv2.COLOR_RGB2GRAY)
    results = reader.readtext(gray)
    ocr_text = "".join([result[1] for result in results])
    return ocr_text

# Run Inference (modified for video)
def run_inference_video(video_path):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for output video

    # Specifying the output path explicitly and ensuring it's created
    output_path = '/content/output.mp4' #  Using a specific path within Colab
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    out = cv2.VideoWriter(output_path, fourcc, 20.0, (width, height))  # Output video file

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)

        # Using result.ims to get the processed frame and writing it to the file
        annotated_frame = results[0].plot()
        out.write(annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Downloading the output video using the output_path variable
    files.download(output_path)  # Download the output video

# Example Usage
video_path = '2103099-uhd_3840_2160_30fps.mp4'  # Replace with the path to your video
run_inference_video(video_path)

ModuleNotFoundError: No module named 'easyocr'

In [6]:
!pip install opencv-python

In [8]:
import os
import torch
import easyocr
import cv2
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors
from PIL import Image
import cv2

# Load trained YOLO model
model = YOLO('best.pt')

# Initialize EasyOCR Reader
reader = easyocr.Reader(['en'])

# OCR Function
def getOCR(im, coors):
    x, y, w, h = int(coors[0]), int(coors[1]), int(coors[2]), int(coors[3])
    cropped = im[y:h, x:w]
    gray = cv2.cvtColor(cropped, cv2.COLOR_RGB2GRAY)
    results = reader.readtext(gray)
    ocr_text = "".join([result[1] for result in results])
    return ocr_text

def run_inference_video(video_path):
    # Read input video
    cap = cv2.VideoCapture(video_path)

    # Check if video opened successfully
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return None

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define codec and create VideoWriter object
    output_path = 'output/output.mp4'

    # Use platform-specific codec
    #if os.name == 'nt':  # Windows
        #fourcc = cv2.VideoWriter_fourcc(*'H264')
    #else:  # Linux/Mac
        #fourcc = cv2.VideoWriter_fourcc(*'avc1')

 # Use platform-specific codec and more robust codec options
    if os.name == 'nt':  # Windows
        fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Try XVID if H264 fails
    else:  # Linux/Mac
        fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Try XVID for better compatibility

         # Ensure directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Check if the codec is supported
    if not cv2.VideoWriter_fourcc(*'XVID'):
        print("XVID codec not supported. Please install the necessary codec libraries.")
        return None

    out = cv2.VideoWriter(
        output_path,
        fourcc,
        fps,
        (width, height),
        isColor=True
    )

    frame_count = 0

    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Convert frame to RGB for YOLO
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Run YOLO inference
            results = model(frame_rgb)

            # Convert back to BGR for OpenCV
            annotated_frame = cv2.cvtColor(results[0].plot(), cv2.COLOR_RGB2BGR)

            # Write frame
            out.write(annotated_frame)

            frame_count += 1
            if frame_count % 30 == 0:  # Print progress every 30 frames
                print(f"Processed {frame_count} frames")

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    finally:
        # Ensure resources are released
        cap.release()
        out.release()
        cv2.destroyAllWindows()

        if os.path.exists(output_path) and os.path.getsize(output_path) > 0:
            print(f"Video saved successfully to {output_path}")
            return output_path
        else:
            print("Error: Output video file is empty or not created")
            return None

# Example Usage
video_path = 'test_video.mp4'
output = run_inference_video(video_path)

Streaming output truncated to the last 5000 lines.

0: 384x640 1 License_Plate, 13.5ms
Speed: 3.4ms preprocess, 13.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 22.6ms
Speed: 3.3ms preprocess, 22.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 13.3ms
Speed: 3.7ms preprocess, 13.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 13.8ms
Speed: 3.4ms preprocess, 13.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 13.8ms
Speed: 3.5ms preprocess, 13.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 20.7ms
Speed: 4.3ms preprocess, 20.7ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 License_Plates, 21.1ms
Speed: 10.2ms preprocess, 21.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Li

In [9]:
os.system("zip -r /content/runs.zip /content/runs")

3072